In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import sys
import re
import numpy as np
import seaborn as sb
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from sklearn.model_selection import train_test_split
from sklearn import feature_extraction, linear_model, model_selection,preprocessing
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.metrics import confusion_matrix,f1_score,recall_score,precision_score,classification_report
from sklearn.feature_selection import chi2
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import Ridge,RidgeClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
import time
from time import sleep

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
nltk.download('punkt')
nltk.download("stopwords")

In [ ]:
dataset = pd.read_csv("/kaggle/input/mobilephonereview/data.csv")

In [ ]:
print("Length of review data: ",len(dataset))
dataset.head()


In [ ]:
# counting total NaN values in each column
dataset.isna().sum()

In [ ]:
# this chart represents top 50 rated words from dataframe 
plt.figure(figsize=(18, 7))
dataset['product'].value_counts()[:10].plot(kind='bar')
plt.ylabel('Count of Rating')
plt.title('Top 10 Rated Mobile')
plt.show()

In [ ]:
def extract_data(dataset):
    x = pd.read_csv(dataset,engine='python')
    x = x[x['lang'] == 'en']
    x.reset_index(inplace=True)
    return x[['score','extract','product']]
dataset = extract_data('/kaggle/input/mobilephonereview/data.csv')

In [ ]:
col_name = ["Rating", "Review", "Mobilephone"]
dataset.columns = col_name

In [ ]:
dataset.head()

In [ ]:
dataset = dataset[['Review','Rating']]
dataset.dropna(inplace=True)
print("Length of dataset after removing missing values: ",len(dataset))
print('')
dataset.head()

In [ ]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
dataset['Review'] = dataset['Review'].apply(lambda x: tokenizer.tokenize(x.lower()))
dataset.head()

In [ ]:
stops = set(stopwords.words("english"))
dataset['Review'] = dataset['Review'].apply(lambda x: [item for item in x if item not in stops])
dataset.head()

In [ ]:
#joining all tokens to proceed further for data modeling
dataset['Review'] = dataset['Review'].apply(' '.join)

In [ ]:
dataset.head()

In [ ]:
dataset.dropna(inplace=True)
np.unique(dataset.Rating)

In [ ]:
def apply_trans(p):
    if p >= 1. and p < 2.:
        return 1
    elif p >= 2. and p < 3.:
        return 2
    elif p >= 3. and p < 4.:
        return 3
    elif p >= 4. and p < 5.:
        return 4
    elif p >= 5. and p < 6.:
        return 5
    elif p >= 6. and p < 7.:
        return 6
    elif p >= 7. and p < 8.:
        return 7
    elif p >= 8. and p < 9.:
        return 8
    elif p >= 9. and p < 10.:
        return 9
    else:
        return 10
        
dataset['Rating'] = dataset['Rating'].apply(lambda x :apply_trans(float(x)))
dataset.reset_index(inplace=True)

In [ ]:
fig = plt.figure(figsize=(9,6))
n, bins, patches = plt.hist(dataset.Rating, bins=40, facecolor='red',alpha = 1)
plt.xlabel('Rating')
plt.ylabel('Counts')
plt.title('Occurrence of each rating in preprocessed data')
fig.patch.set_facecolor('white')
plt.show()
# width of bars are defined using value of bin

In [ ]:
#average value of all the ratings 
print("Average rating: ",dataset['Rating'].mean())
#median value of ratings values
print("Median value: ",dataset['Rating'].median())

In [ ]:
new_dataset = dataset

In [ ]:
#occurance of each rating in the column
new_dataset['Rating'].value_counts()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(new_dataset['Review'], new_dataset['Rating'], test_size=0.2) # 80% training and 20% test

In [ ]:
vectorizer = TfidfVectorizer()
transformer = TfidfTransformer()

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

X_train_counts = transformer.fit_transform(X_train)
X_test_counts = transformer.fit_transform(X_test)


In [ ]:
model_linear_SVC = LinearSVC().fit(X_train,y_train.astype('int'))
y_predicted_linear_SVC = model_linear_SVC.predict(X_test)
accuracy = accuracy_score(y_test.astype('int'),y_predicted_linear_SVC) * float(100)
accuracy_linear_svc = str(accuracy)
print('Testing Accuracy for Linear Support vector machine is: '+accuracy_linear_svc+' %')

In [ ]:
print("Confusion matrix for Linear Support vector machine:\n")
fig= plt.figure(figsize=(8,6))
ax = fig.add_axes([1,1,1,1])
matrix = confusion_matrix(y_test.astype('int'), y_predicted_linear_SVC)
sb.heatmap(matrix/np.sum(matrix), square=True, annot=True, cmap="Reds")
ax.set_xlabel('Actual rating')
ax.set_ylabel('Predicted rating')

In [ ]:
model_ridge_class = RidgeClassifier().fit(X_train_counts, y_train.astype('int'))
y_predicted = model_ridge_class.predict(X_test_counts)
y_predicted_ridge_class = np.round(y_predicted)
accuracy = accuracy_score(y_test,y_predicted_ridge_class) * float(100)
ridge_class_accuracy = str(accuracy)
print('Testing Accuracy on ridge classifier model is: '+ridge_class_accuracy+' %')

In [ ]:
print("Confusion matrix for Ridge Classifier:\n")
fig= plt.figure(figsize=(10,6))
ax = fig.add_axes([1,1,1,1])
matrix = confusion_matrix(y_test.astype('int'), y_predicted_ridge_class)
sb.heatmap(matrix/np.sum(matrix), square=True, annot=True, cmap="Blues")
ax.set_xlabel('Actual rating')
ax.set_ylabel('Predicted rating')


In [ ]:
model_mNB = MultinomialNB().fit(X_train_counts, y_train.astype('int'))
y_predicted_mNB = model_mNB.predict(X_test_counts)
accuracy = accuracy_score(y_test.astype('int'),y_predicted_mNB) * float(100)
accuracy_mNB = str(accuracy)
print('Testing Accuracy on multinomial naive bayes model is: '+accuracy_mNB+' %')

In [ ]:
#confusion matrix for what output we got
print("Confusion matrix for Multinomial Naive Bayes classifier:\n")
fig= plt.figure(figsize=(15,9))
ax = fig.add_axes([1,1,1,1])
matrix = confusion_matrix(y_test.astype('int'), y_predicted_mNB)
sb.heatmap(matrix/np.sum(matrix), square=True, annot=True)
ax.set_xlabel('Actual rating')
ax.set_ylabel('Predicted rating')

In [ ]:
labels = ['0.0','1.0','2.0','3.0','4.0','5.0','6.0','7.0','8.0','9.0']
p = classification_report(y_test.astype('int'), y_predicted_linear_SVC , target_names=labels,zero_division=1)
print("Report for Linear SVC:\n ")
print(p)
print('')

In [ ]:
p = classification_report(y_test.astype('int'), y_predicted_ridge_class, target_names=labels,zero_division=1,)
print("Report for Ridge regression:\n ")
print(p)

In [ ]:
p = classification_report(y_test.astype('int'), y_predicted_mNB, target_names=labels,zero_division=1,)
print("Report for KNN:\n ")
print(p)

In [ ]:
acc1 = float(accuracy_linear_svc)
acc2 = float(ridge_class_accuracy)
acc3 = float(accuracy_mNB)

In [ ]:
acc=[]
acc.append(acc1)
acc.append(acc2)
acc.append(acc3)

In [ ]:
fig = plt.figure(figsize=(6,3))
labels = ['Linear SVC','Ridge','MNB']
ax = fig.add_axes([0, 0, 1, 1])
ax.bar(labels,acc)
plt.ylim(0,80)
plt.show()

In [ ]:
def accuracy_linear_svc(smt):
  smt=[smt] 
  smt = vectorizer.transform(smt) 
  smt = transformer.fit_transform(smt) #transforming input string vector
  prediction = np.round(model_linear_SVC.predict(smt)) #using ridge classifier trained model to predict rating for what user has gave input
  return prediction   